In [1]:
import sys
sys.path.append("../../")

In [2]:
from DataLoader.dataset import FoodRatingsDataset
from torch.utils.data import DataLoader

from tqdm.notebook import tqdm
import math
import os
import pandas as pd
import numpy as np

## Load data

In [3]:
n_users = 226570
n_items = 231637
# n_item_features = 120910
# n_item_features = 231647
# n_item_features = 231637
n_item_features = 352547

# batch_size = 256
batch_size = 4086

device = "cuda"

train_dataloader = DataLoader(
    FoodRatingsDataset(
        "../../Preprocessing/processed_dataframes/train.csv",
#         "../../Preprocessing/processed_dataframes/val.csv",
        "../../Preprocessing/processed_dataframes/sorted_recipes.csv",
        n_users,
        n_items,
        has_rating_column=True,
    ), 
    batch_size=batch_size,
    num_workers=8,
    shuffle=False,
)

val_dataloader = DataLoader(
    FoodRatingsDataset(
        "../../Preprocessing/processed_dataframes/val.csv",
#         "../../Preprocessing/processed_dataframes/test.csv",
        "../../Preprocessing/processed_dataframes/sorted_recipes.csv",
        n_users,
        n_items,
        has_rating_column=True,
    ), 
    batch_size=batch_size,
    num_workers=2,
    shuffle=False,
)

test_dataloader = DataLoader(
    FoodRatingsDataset(
        "../../Preprocessing/processed_dataframes/test.csv",
        "../../Preprocessing/processed_dataframes/sorted_recipes.csv",
        n_users,
        n_items,
        has_rating_column=True,
    ), 
    batch_size=batch_size,
    num_workers=10,
    shuffle=False,
)

In [4]:
import scipy.sparse as sparse

In [5]:
preparations_tfidf = sparse.load_npz("../../Preprocessing/objects/preparations_tfidf.npz")
tags_matrix = sparse.load_npz("../../Preprocessing/objects/tags_matrix.npz")

## Train

In [6]:
import torch

arange = torch.arange(batch_size, device=device)
ones = torch.ones(batch_size, dtype=torch.float32, device=device)

In [7]:
from train_functions import fit, predict

In [ ]:
losses_fit = []

for run_number, (k_gmf, k_mlp, layers, lr, epochs) in tqdm(enumerate([
#     (8, 16, [16, 8], 0.0001, 75),
#     (8, 16, [16, 8], 0.0006, 50),
#     (8, 16, [16, 8], 0.0008, 50),
#     (16, 16, [16, 8], 0.001, 50),
#     (16, 32, [32, 16, 8], 0.001, 50),
#     (32, 16, [16, 8], 0.0008, 30),
#     (32, 16, [16, 8], 0.0001, 50),
#     (32, 16, [16, 8], 0.001, 50),
#     (4, 8, [8], 0.0005, 50),
    (8, 8, [16,32,16,8], 0.0001, 100)
#     (32, 16, [16, 8], 0.005, 50),
#     (32, 16, [16, 8], 0.01, 50),
#     (32, 32, [32, 16, 8], 0.01, 50),
#     (32, 32, [32, 16, 8], 0.00001, 100),
#     (32, 32, [32, 16, 8], 0.0001, 100),
])):
    print(f"{k_gmf=} {k_mlp=} {layers=} {lr=} {epochs=}")
    losses_fit.append(
        fit(
            train_dataloader, 
            val_dataloader,
            n_users,
            n_items,
            n_item_features,
            k_gmf,
            k_mlp,
            layers,
            preparations_tfidf,
            tags_matrix,
            arange,
            ones,
            0.5,
            lr,
            epochs,
            weight_path="/home/nubol23/Documents/Project/NCF_dot",
            run_number=run_number,
            random_state=3,
            show_loss=True,
            device="cuda",
        )
    )

0it [00:00, ?it/s]

k_gmf=8 k_mlp=8 layers=[16, 32, 16, 8] lr=0.0001 epochs=100
Adam (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    eps: 1e-08
    lr: 0.0001
    maximize: False
    weight_decay: 0
)


  0%|          | 0/100 [00:00<?, ?it/s]

epoch: 1, train_loss: 4.659649948760794, val_loss: 4.616104198026074
epoch: 2, train_loss: 4.507797676129347, val_loss: 4.363673293207191
epoch: 3, train_loss: 4.054220587677243, val_loss: 3.523313020969687
epoch: 4, train_loss: 2.404868172596289, val_loss: 1.6729513822772355
epoch: 5, train_loss: 1.6282175643496144, val_loss: 1.5882686502781338
epoch: 6, train_loss: 1.5490523246063561, val_loss: 1.5170434004720714
epoch: 7, train_loss: 1.4791022742006157, val_loss: 1.4526733593625027
epoch: 8, train_loss: 1.415246823285051, val_loss: 1.3934911081683468
epoch: 9, train_loss: 1.3563350479418983, val_loss: 1.338782846572298
epoch: 10, train_loss: 1.3018943719950622, val_loss: 1.288311022602631
epoch: 11, train_loss: 1.25181650123181, val_loss: 1.2421228363559755
epoch: 12, train_loss: 1.206213177525969, val_loss: 1.2004037560754799
epoch: 13, train_loss: 1.1652852529816609, val_loss: 1.1633767166216946
epoch: 14, train_loss: 1.1291796804635046, val_loss: 1.1311222420147697
epoch: 15, tra